In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit


### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00


In [2]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

In [2]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [3]:
# @title Initialize Vertex AI with Staging Bucket.
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Agent on adk

In [24]:
#@title Build a Agent with adk.
from google.adk.agents import Agent
from google.adk.tools import google_search

agent = Agent(
    name="search_assistant",
    model=MODEL_NAME,
    instruction="You are a helpful assistant. Answer user questions using Google Search when needed.",
    description="An assistant that can search the web.",
    tools=[google_search] # Use Google search tool
)


In [25]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
   user_id="shins",  # Required
   message="What is the exchange rate from US dollars to Korean currency today ?",
):
   print(event)
   print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': 'As of April 16, 2025, here are the approximate exchange rates from US dollars (USD) to South Korean Won (KRW):\n\n*   **Wise:** 1 USD = 1,425.980 KRW\n*   **XE.com:** 1 USD = 1,428.44 KRW\n*   **Trading Economics:** 1 USD = 1,427.4600 KRW'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'xe.com', 'title': 'xe.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJlHQr2SOA-hMT2J1ppU3hy61JS88XZPzqUhEeJh5rnaMly0kR_8quAo5nw3TIjfdRQK_gXOy4-iNUao__-896MZQ0RoQ90MJKAikJjV7ei3GA1i-MuqzACWBjxozcgQd9cNzwxrf79Ml5jks8P3EeGb1O3sJyzlCRUOJOE9LDtO4hOh9U='}}, {'web': {'domain': 'tradingeconomics.com', 'title': 'tradingeconomics.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIYXxtWC8WDXhhyZZyW4CN_xK40xaoyu2eIwKbZ9b8fc_mgD-CsVJtjaU6ujGEbI2qRoM0AheS8qY_fIr7HetJviCW8dp8nhqI2tywqAbuO5w6zXWjXNL2Ll93hajXMm9xElreRnawzxP7Q-Y4='}}], 'grounding_supports': [{'confidence_scores': [0.7

## Use a Tool

In [26]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "KRW" (KRW).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [27]:
get_exchange_rate(currency_from="USD", currency_to="KRW")


{'amount': 1.0, 'base': 'USD', 'date': '2025-04-15', 'rates': {'KRW': 1426.85}}

In [28]:
from google.adk.agents import Agent

agent = Agent(
    model=MODEL_NAME,
    name='currency_exchange_agent',
    tools=[get_exchange_rate],
)

In [29]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(agent=agent)

for event in app.stream_query(
    user_id="USER_ID",
    message="What is the exchange rate from US dollars to Korean Won on 2025-04-15?",
):
   print(event)


{'content': {'parts': [{'function_call': {'id': 'adk-34dd171d-1978-49a6-aaad-0adde354ba0e', 'args': {'currency_to': 'KRW', 'currency_from': 'USD', 'currency_date': '2025-04-15'}, 'name': 'get_exchange_rate'}}], 'role': 'model'}, 'invocation_id': 'e-cf88c66a-c78f-4164-9672-d6f8202e2cfa', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'lNFMriSy', 'timestamp': 1744771644.30084}
{'content': {'parts': [{'function_response': {'id': 'adk-34dd171d-1978-49a6-aaad-0adde354ba0e', 'name': 'get_exchange_rate', 'response': {'amount': 1.0, 'base': 'USD', 'date': '2025-04-15', 'rates': {'KRW': 1426.85}}}}], 'role': 'user'}, 'invocation_id': 'e-cf88c66a-c78f-4164-9672-d6f8202e2cfa', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '46lVGNpp', 'timestamp': 1744771645.923166}
{'content': {'parts': [{'text': 'The excha

## Manage sessions

In [30]:
def session_service_builder():
  from google.adk.sessions import InMemorySessionService

  return InMemorySessionService()

In [31]:
from vertexai.preview.reasoning_engines import AdkApp

app = AdkApp(
   agent=agent,                                      # Required.
   session_service_builder=session_service_builder,  # Optional.
)

In [32]:
session = app.create_session(user_id="shins")


In [33]:
app.list_sessions(user_id="shins")


ListSessionsResponse(sessions=[Session(id='4a7b8094-6f0c-4f15-87db-2652e5f1f98f', app_name='default-app-name', user_id='shins', state={}, events=[], last_update_time=1744771783.274824)])

In [35]:
session = app.get_session(user_id="shins", session_id="4a7b8094-6f0c-4f15-87db-2652e5f1f98f")
print(session)

id='4a7b8094-6f0c-4f15-87db-2652e5f1f98f' app_name='default-app-name' user_id='shins' state={} events=[] last_update_time=1744771783.274824


In [37]:
for event in app.stream_query(
    user_id="shins",
    session_id='4a7b8094-6f0c-4f15-87db-2652e5f1f98f', # Optional. you can pass in the session_id when querying the agent
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-59fc05d9-7fa6-4a20-ac8b-6e1711c4440d', 'args': {'currency_date': '2025-04-15', 'currency_to': 'KRW', 'currency_from': 'USD'}, 'name': 'get_exchange_rate'}}], 'role': 'model'}, 'invocation_id': 'e-3acbfab0-eb00-4add-8b3a-c363a41442a5', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'TEWxV2yI', 'timestamp': 1744774002.339464}
{'content': {'parts': [{'function_response': {'id': 'adk-59fc05d9-7fa6-4a20-ac8b-6e1711c4440d', 'name': 'get_exchange_rate', 'response': {'amount': 1.0, 'base': 'USD', 'date': '2025-04-15', 'rates': {'KRW': 1426.85}}}}], 'role': 'user'}, 'invocation_id': 'e-3acbfab0-eb00-4add-8b3a-c363a41442a5', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'U5Vw0LNq', 'timestamp': 1744774004.216273}
{'content': {'parts': [{'text': 'OK. The 

In [38]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    app,
    display_name="currency agent with adk",
    gcs_dir_name = "ai-agent-adk",
    description="This is a simple agent with adk on agent engine.",
    requirements=[
        "google-adk",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.88.0', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-adk', 'cloudpickle==3.0']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-adk/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-adk/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/4968182872524455936/operations/4520339245226786816
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=ai-hangsik
INFO:vertexai.agent_engines:AgentEngine created. Resource

In [43]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-15?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-01f0e6e4-27a5-4502-b108-54a6b3d3c5fb', 'args': {'currency_from': 'USD', 'currency_date': '2025-04-15', 'currency_to': 'KRW'}, 'name': 'get_exchange_rate'}}], 'role': 'model'}, 'invocation_id': 'e-d3391f68-64a4-4cda-9914-f317bfbbcaaf', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'U0OP6C0t', 'timestamp': 1744774387.444382}
{'content': {'parts': [{'function_response': {'id': 'adk-01f0e6e4-27a5-4502-b108-54a6b3d3c5fb', 'name': 'get_exchange_rate', 'response': {'amount': 1.0, 'base': 'USD', 'date': '2025-04-15', 'rates': {'KRW': 1426.85}}}}], 'role': 'user'}, 'invocation_id': 'e-d3391f68-64a4-4cda-9914-f317bfbbcaaf', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'bPmYEOd6', 'timestamp': 1744774388.694623}
{'content': {'parts': [{'text': 'OK. The exc

## Agent Management

In [44]:
# @title Helper function to manage reasoning engine.

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview

#----------------------------------------
def get_agent_engine(display_name:str):
  try:
    for agent in agent_engines.list():
      if agent.display_name == display_name:
        return agent_engines.get(agent.name)

      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def show_agents():
  """
  List reasoning engines.
  """

  try:
    if not agent_engines.list():
      print("No reasoning engines")

    for idx, agent in enumerate(agent_engines.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{agent.display_name}] \n\tName [{agent.name}] \n\tCreation Time [{agent.create_time}] \n\tResource Name [{agent.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def delete_agent(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = agent_engines.get(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [45]:
show_agents()

Agent 0: 
	Display Name [currency agent with adk] 
	Name [4968182872524455936] 
	Creation Time [2025-04-16 03:28:46.545118+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/4968182872524455936]

Agent 1: 
	Display Name [currency agent] 
	Name [712483534798848] 
	Creation Time [2025-04-16 00:35:51.892002+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/712483534798848]



In [46]:
get_agent_engine(display_name="currency agent with adk")


resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/4968182872524455936

In [ ]:
delete_agent(name="5350988840850948096")

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/721521243942/locations/us-central1/operations/964465864441069568
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/5350988840850948096


Deleted 5350988840850948096


In [49]:
# @title Query from remote engine.

for event in remote_agent.stream_query(
    user_id="shins",
    message="What is the exchange rate from US dollars to Korean Won currency on 2025-04-14?",
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-f499ab12-4f44-4a55-9acd-e7b9cb18b608', 'args': {'currency_from': 'USD', 'currency_to': 'KRW', 'currency_date': '2025-04-14'}, 'name': 'get_exchange_rate'}}], 'role': 'model'}, 'invocation_id': 'e-01132549-1b02-4e09-987f-5fd735560c23', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'ikaISZn6', 'timestamp': 1744774889.907666}
{'content': {'parts': [{'function_response': {'id': 'adk-f499ab12-4f44-4a55-9acd-e7b9cb18b608', 'name': 'get_exchange_rate', 'response': {'amount': 1.0, 'base': 'USD', 'date': '2025-04-14', 'rates': {'KRW': 1421.54}}}}], 'role': 'user'}, 'invocation_id': 'e-01132549-1b02-4e09-987f-5fd735560c23', 'author': 'currency_exchange_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'vpR5M2kE', 'timestamp': 1744774891.103188}
{'content': {'parts': [{'text': 'OK. The exc

## End of notebook